# Machine Learning with PyTorch and Scikit-Learn  
# -- Code Examples

## Obtaining and preparing the MNIST dataset

In [13]:
from sklearn.datasets import fetch_openml


X, y = fetch_openml('mnist_784', version=1, return_X_y=True)
X = X.values
y = y.astype(int).values

print(X.shape)
print(y.shape)

(70000, 784)
(70000,)


Normalize to [-1, 1] range:

In [14]:
X = ((X / 255.) - .5) * 2

In [15]:
from sklearn.model_selection import train_test_split


# X_temp, X_test, y_temp, y_test = train_test_split(
#     X, y, test_size=10000, random_state=123, stratify=y)

# X_train, X_valid, y_train, y_valid = train_test_split(
#     X_temp, y_temp, test_size=5000, random_state=123, stratify=y_temp)

X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.7,test_size=0.3)

# optional to free up some memory by deleting non-used arrays:
# del X_temp, y_temp, X, y

<br>
<br>

## Implementing a multi-layer perceptron

In [16]:
import numpy as np
from math import exp

In [17]:
##########################
# MODEL
##########################

def sigmoid(z):
    return 1. / (1. + np.exp(-z))


def softmax(x):
    exps = np.exp(X - np.max(X))
    return exps / np.sum(exps)


def int_to_onehot(y, num_labels):

    ary = np.zeros((y.shape[0], num_labels))
    for i, val in enumerate(y):
        ary[i, val] = 1

    return ary


class NeuralNetMLP2:

    def __init__(self, num_features, num_hidden, num_classes, random_seed=123, softmaxUse=False):
        super().__init__()

        num_hidden1 = num_hidden
        num_hidden2 = num_hidden
        self.num_classes = num_classes

        # hidden first
        rng = np.random.RandomState(random_seed)

        self.weight_h = rng.normal(
            loc=0.0, scale=0.1, size=(num_hidden1, num_features))
        self.bias_h = np.zeros(num_hidden1)

        # hidden second
        rng = np.random.RandomState(random_seed)

        self.weight_h2 = rng.normal(
            loc=0.0, scale=0.1, size=(num_hidden2, num_hidden1))
        self.bias_h2 = np.zeros(num_hidden2)

        # output
        self.weight_out = rng.normal(
            loc=0.0, scale=0.1, size=(num_classes, num_hidden2))
        self.bias_out = np.zeros(num_classes)
        self.softmaxUse = softmaxUse

    def forward(self, x):
        # Hidden layer
        # input dim: [n_examples, n_features] dot [n_hidden, n_features].T
        # output dim: [n_examples, n_hidden]
        z_h1 = np.dot(x, self.weight_h.T) + self.bias_h
        a_h1 = sigmoid(z_h1)

        # Hidden layer 2
        z_h2 = np.dot(a_h1, self.weight_h2.T) + self.bias_h2
        a_h2 = sigmoid(z_h2)

        # Output layer
        # input dim: [n_examples, n_hidden] dot [n_classes, n_hidden].T
        # output dim: [n_examples, n_classes]
        z_out = np.dot(a_h2, self.weight_out.T) + self.bias_out
        a_out = sigmoid(z_out)

        return a_h1, a_h2, a_out

    def backward(self, x, a_h1, a_h2, a_out, y):

        #########################
        # Output layer weights
        #########################

        # onehot encoding
        y_onehot = int_to_onehot(y, self.num_classes)

        # Part 1: dLoss/dOutWeights
        # = dLoss/dOutAct * dOutAct/dOutNet * dOutNet/dOutWeight
        # where DeltaOut = dLoss/dOutAct * dOutAct/dOutNet
        # for convenient re-use

        # input/output dim: [n_examples, n_classes]
        d_loss__d_a_out = 2.*(a_out - y_onehot) / y.shape[0]

        # input/output dim: [n_examples, n_classes]
        d_a_out__d_z_out = a_out * (1. - a_out)  # sigmoid derivative

        # output dim: [n_examples, n_classes]
        # "delta (rule) placeholder"
        delta_out = d_loss__d_a_out * d_a_out__d_z_out

        # gradient for output weights

        # [n_examples, n_hidden]
        d_z_out__dw_out = a_h2

        # input dim: [n_classes, n_examples] dot [n_examples, n_hidden]
        # output dim: [n_classes, n_hidden]
        d_loss__dw_out = np.dot(delta_out.T, d_z_out__dw_out)
        d_loss__db_out = np.sum(delta_out, axis=0)

        #################################
        # Part 2: dLoss/dHiddenWeights
        # = DeltaOut * dOutNet/dHiddenAct * dHiddenAct/dHiddenNet * dHiddenNet/dWeight

        # [n_classes, n_hidden]
        d_z_out__a_h = self.weight_out

        # output dim: [n_examples, n_hidden]
        d_loss__a_h = np.dot(delta_out, d_z_out__a_h)

        # [n_examples, n_hidden]
        d_a_h__d_z_h = a_h2 * (1. - a_h2)  # sigmoid derivative

        # [n_examples, n_features]
        d_z_h__d_w_h = a_h1

        # output dim: [n_hidden, n_features]
        d_loss__d_w_h2 = np.dot((d_loss__a_h * d_a_h__d_z_h).T, d_z_h__d_w_h)
        d_loss__d_b_h2 = np.sum((d_loss__a_h * d_a_h__d_z_h), axis=0)

    # ME ADD
        delta_out = d_loss__a_h * d_a_h__d_z_h

        #################################
        # Part 3: dLoss/dHiddenWeights
        # = DeltaOut * dOutNet/dHiddenAct * dHiddenAct/dHiddenNet * dHiddenNet/dWeight

        # [n_classes, n_hidden]
        d_z_out__a_h = self.weight_h2

        # output dim: [n_examples, n_hidden]
        d_loss__a_h = np.dot(delta_out, d_z_out__a_h)

        # [n_examples, n_hidden]
        d_a_h__d_z_h = a_h1 * (1. - a_h1)  # sigmoid derivative

        # [n_examples, n_features]
        d_z_h__d_w_h = x

        # output dim: [n_hidden, n_features]
        d_loss__d_w_h1 = np.dot((d_loss__a_h * d_a_h__d_z_h).T, d_z_h__d_w_h)
        d_loss__d_b_h1 = np.sum((d_loss__a_h * d_a_h__d_z_h), axis=0)

        return (d_loss__dw_out, d_loss__db_out,
                d_loss__d_w_h1, d_loss__d_b_h1,
                d_loss__d_w_h2, d_loss__d_b_h2)


In [18]:
model = NeuralNetMLP2(num_features=28*28,
                     num_hidden=500,
                     num_classes=10)

## Coding the neural network training loop

Defining data loaders:

In [19]:
import numpy as np

num_epochs = 50
minibatch_size = 100


def minibatch_generator(X, y, minibatch_size):
    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)

    for start_idx in range(0, indices.shape[0] - minibatch_size 
                           + 1, minibatch_size):
        batch_idx = indices[start_idx:start_idx + minibatch_size]
        
        yield X[batch_idx], y[batch_idx]

        
# iterate over training epochs
for i in range(num_epochs):

    # iterate over minibatches
    minibatch_gen = minibatch_generator(
        X_train, y_train, minibatch_size)
    
    for X_train_mini, y_train_mini in minibatch_gen:

        break
        
    break
    
print(X_train_mini.shape)
print(y_train_mini.shape)

(100, 784)
(100,)


Defining a function to compute the loss and accuracy

In [20]:
def mse_loss(targets, probas, num_labels=10):
    onehot_targets = int_to_onehot(targets, num_labels=num_labels)
    return np.mean((onehot_targets - probas)**2)


def accuracy(targets, predicted_labels):
    return np.mean(predicted_labels == targets)


def predict(model, X):
    _, _, probas = model.forward(X)
    predicted_labels = np.argmax(probas, axis=1)
    return predicted_labels

# _, _, probas = model.forward(X_valid)
# mse = mse_loss(y_valid, probas)

# predicted_labels = np.argmax(probas, axis=1)
# acc = accuracy(y_valid, predicted_labels)

# print(f'Initial validation MSE: {mse:.1f}')
# print(f'Initial validation accuracy: {acc*100:.1f}%')


In [21]:
def compute_mse_and_acc(nnet, X, y, num_labels=10, minibatch_size=100):
    mse, correct_pred, num_examples = 0., 0, 0
    minibatch_gen = minibatch_generator(X, y, minibatch_size)
        
    for i, (features, targets) in enumerate(minibatch_gen):

        _,_, probas = nnet.forward(features)
        predicted_labels = np.argmax(probas, axis=1)
        
        onehot_targets = int_to_onehot(targets, num_labels=num_labels)
        loss = np.mean((onehot_targets - probas)**2)
        correct_pred += (predicted_labels == targets).sum()
        
        num_examples += targets.shape[0]
        mse += loss

    mse = mse/i
    acc = correct_pred/num_examples
    return mse, acc

In [22]:
mse, acc = compute_mse_and_acc(model, X_test, y_test)
print(f'Initial valid MSE: {mse:.1f}')
print(f'Initial valid accuracy: {acc*100:.1f}%')

Initial valid MSE: 0.4
Initial valid accuracy: 11.5%


In [23]:
def train(model, X_train, y_train, X_valid, y_valid, num_epochs,
          learning_rate=0.1):

    epoch_loss = []
    epoch_train_acc = []
    epoch_valid_acc = []

    for e in range(num_epochs):

        # iterate over minibatches
        minibatch_gen = minibatch_generator(
            X_train, y_train, minibatch_size)

        for X_train_mini, y_train_mini in minibatch_gen:

            #### Compute outputs ####
            a_h1, a_h2, a_out = model.forward(X_train_mini)

            #### Compute gradients ####
            d_loss__d_w_out, d_loss__d_b_out, d_loss__d_w_h1, d_loss__d_b_h1, d_loss__d_w_h2, d_loss__d_b_h2 = \
                model.backward(X_train_mini, a_h1, a_h2, a_out, y_train_mini)

            #### Update weights ####
            model.weight_h -= learning_rate * d_loss__d_w_h1
            model.bias_h -= learning_rate * d_loss__d_b_h1

            model.weight_h2 -= learning_rate * d_loss__d_w_h2
            model.bias_h2 -= learning_rate * d_loss__d_b_h2
            
            model.weight_out -= learning_rate * d_loss__d_w_out
            model.bias_out -= learning_rate * d_loss__d_b_out

        #### Epoch Logging ####
        train_mse, train_acc = compute_mse_and_acc(model, X_train, y_train)
        valid_mse, valid_acc = compute_mse_and_acc(model, X_valid, y_valid)
        train_acc, valid_acc = train_acc*100, valid_acc*100
        epoch_train_acc.append(train_acc)
        epoch_valid_acc.append(valid_acc)
        epoch_loss.append(train_mse)
        print(f'Epoch: {e+1:03d}/{num_epochs:03d} '
              f'| Train MSE: {train_mse:.2f} '
              f'| Train Acc: {train_acc:.2f}% '
              f'| Valid Acc: {valid_acc:.2f}%')

    return epoch_loss, epoch_train_acc, epoch_valid_acc


In [24]:
np.random.seed(123) # for the training set shuffling

epoch_loss, epoch_train_acc, epoch_valid_acc = train(
    model, X_train, y_train, X_test, y_test,
    num_epochs=50, learning_rate=0.1)

Epoch: 001/050 | Train MSE: 0.03 | Train Acc: 84.39% | Valid Acc: 84.45%
Epoch: 002/050 | Train MSE: 0.02 | Train Acc: 87.72% | Valid Acc: 87.79%
Epoch: 003/050 | Train MSE: 0.02 | Train Acc: 89.04% | Valid Acc: 89.12%
Epoch: 004/050 | Train MSE: 0.02 | Train Acc: 89.93% | Valid Acc: 89.90%
Epoch: 005/050 | Train MSE: 0.02 | Train Acc: 90.67% | Valid Acc: 90.67%
Epoch: 006/050 | Train MSE: 0.02 | Train Acc: 91.08% | Valid Acc: 91.10%
Epoch: 007/050 | Train MSE: 0.02 | Train Acc: 91.34% | Valid Acc: 91.36%
Epoch: 008/050 | Train MSE: 0.01 | Train Acc: 91.95% | Valid Acc: 91.79%
Epoch: 009/050 | Train MSE: 0.01 | Train Acc: 92.20% | Valid Acc: 92.06%
Epoch: 010/050 | Train MSE: 0.01 | Train Acc: 92.55% | Valid Acc: 92.44%
Epoch: 011/050 | Train MSE: 0.01 | Train Acc: 92.80% | Valid Acc: 92.51%
Epoch: 012/050 | Train MSE: 0.01 | Train Acc: 92.97% | Valid Acc: 92.88%
Epoch: 013/050 | Train MSE: 0.01 | Train Acc: 93.45% | Valid Acc: 93.15%
Epoch: 014/050 | Train MSE: 0.01 | Train Acc: 93.64

In [25]:
test_mse, test_acc = compute_mse_and_acc(model, X_test, y_test)
print(f'Test accuracy: {test_acc*100:.2f}%')

Test accuracy: 96.50%
